In [24]:
%matplotlib widget

from __future__ import annotations

import re
from collections import defaultdict
from dataclasses import dataclass, field
from itertools import permutations, product
from math import inf
from random import choice

import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import numpy.typing as npt
from mpl_toolkits.mplot3d import axes3d
from numpy import int_, object_
from numpy.typing import NDArray
from test_utilities import test
from util import *

COLORS = list(mcolors.CSS4_COLORS.keys())

<link href="style.css" rel="stylesheet"></link>
<article class="day-desc"><h2>--- Day 21: Monkey Math ---</h2><p>The <a href="day11.ipynb">monkeys</a> are back! You're worried they're going to try to steal your stuff again, but it seems like they're just holding their ground and making various monkey noises at you.</p>
<p>Eventually, one of the elephants realizes you don't speak monkey and comes over to interpret. As it turns out, they overheard you talking about trying to find the grove; they can show you a shortcut if you answer their <em>riddle</em>.</p>
<p>Each monkey is given a <em>job</em>: either to <em>yell a specific number</em> or to <em>yell the result of a math operation</em>. All of the number-yelling monkeys know their number from the start; however, the math operation monkeys need to wait for two other monkeys to yell a number, and those two other monkeys might <em>also</em> be waiting on other monkeys.</p>
<p>Your job is to <em>work out the number the monkey named <code>root</code> will yell</em> before the monkeys figure it out themselves.</p>
<p>For example:</p>
<pre><code>root: pppw + sjmn
dbpl: 5
cczh: sllz + lgvd
zczc: 2
ptdq: humn - dvpt
dvpt: 3
lfqf: 4
humn: 5
ljgn: 2
sjmn: drzm * dbpl
sllz: 4
pppw: cczh / lfqf
lgvd: ljgn * ptdq
drzm: hmdt - zczc
hmdt: 32
</code></pre>
<p>Each line contains the name of a monkey, a colon, and then the job of that monkey:</p>
<ul>
<li>A lone number means the monkey's job is simply to yell that number.</li>
<li>A job like <code>aaaa + bbbb</code> means the monkey waits for monkeys <code>aaaa</code> and <code>bbbb</code> to yell each of their numbers; the monkey then yells the sum of those two numbers.</li>
<li><code>aaaa - bbbb</code> means the monkey yells <code>aaaa</code>'s number minus <code>bbbb</code>'s number.</li>
<li>Job <code>aaaa * bbbb</code> will yell <code>aaaa</code>'s number multiplied by <code>bbbb</code>'s number.</li>
<li>Job <code>aaaa / bbbb</code> will yell <code>aaaa</code>'s number divided by <code>bbbb</code>'s number.</li>
</ul>
<p>So, in the above example, monkey <code>drzm</code> has to wait for monkeys <code>hmdt</code> and <code>zczc</code> to yell their numbers. Fortunately, both <code>hmdt</code> and <code>zczc</code> have jobs that involve simply yelling a single number, so they do this immediately: <code>32</code> and <code>2</code>. Monkey <code>drzm</code> can then yell its number by finding <code>32</code> minus <code>2</code>: <code><em>30</em></code>.</p>
<p>Then, monkey <code>sjmn</code> has one of its numbers (<code>30</code>, from monkey <code>drzm</code>), and already has its other number, <code>5</code>, from <code>dbpl</code>. This allows it to yell its own number by finding <code>30</code> multiplied by <code>5</code>: <code><em>150</em></code>.</p>
<p>This process continues until <code>root</code> yells a number: <code><em>152</em></code>.</p>
<p>However, your actual situation involves <span title="Advent of Code 2022: Now With Considerably More Monkeys">considerably more monkeys</span>. <em>What number will the monkey named <code>root</code> yell?</em></p>
</article>


In [25]:
from operator import add, eq, floordiv, mul, sub


tests = [
    {
        "name": "Example 1",
        "s": """
            root: pppw + sjmn
            dbpl: 5
            cczh: sllz + lgvd
            zczc: 2
            ptdq: humn - dvpt
            dvpt: 3
            lfqf: 4
            humn: 5
            ljgn: 2
            sjmn: drzm * dbpl
            sllz: 4
            pppw: cczh / lfqf
            lgvd: ljgn * ptdq
            drzm: hmdt - zczc
            hmdt: 32
        """,
        "expected": 152,
    },
]


class Monkeys(Str):
    OPS = {"+": add, "-": sub, "/": floordiv, "*": mul}

    def __init__(self, s: str) -> None:
        self.g = self._parse(s)

    @classmethod
    def _parse(cls, s: str) -> dict[dict]:
        g = {}
        for l in s.strip().splitlines():
            node, *rest = re.findall(r"[a-z|+-/*//\d]+", l)
            if len(rest) == 1:
                g[node] = {"value": int(rest[0])}
            else:
                g[node] = {
                    "dependencies": (rest[0], rest[-1]),
                    "operation": rest[1],
                }

        return g

    def get(self, node: str) -> int:
        if "value" in self.g[node]:
            return self.g[node]["value"]
        left, right = self.g[node]["dependencies"]
        self.g[node]["value"] = self.OPS[self.g[node]["operation"]](
            self.get(left), self.get(right)
        )
        return self.g[node]["value"]


@test(tests=tests)
def test_part_I(s: str) -> int:
    return Monkeys(s).get("root")


Test Example 1 passed, for test_part_I.
Success


In [26]:
with open("../input/day21.txt") as f:
    puzzle = f.read()

print(f"Part I: {Monkeys(puzzle).get("root"):,}")

Part I: 158,731,561,459,602


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>158731561459602</code>.</p><p class="day-success">The first half of this puzzle is complete! It provides one gold star: *</p>


<link href="style.css" rel="stylesheet"></link>
<article class="day-desc"><h2 id="part2">--- Part Two ---</h2><p>Due to some kind of monkey-elephant-human mistranslation, you seem to have misunderstood a few key details about the riddle.</p>
<p>First, you got the wrong job for the monkey named <code>root</code>; specifically, you got the wrong math operation. The correct operation for monkey <code>root</code> should be <code>=</code>, which means that it still listens for two numbers (from the same two monkeys as before), but now checks that the two numbers <em>match</em>.</p>
<p>Second, you got the wrong monkey for the job starting with <code>humn:</code>. It isn't a monkey - it's <em>you</em>. Actually, you got the job wrong, too: you need to figure out <em>what number you need to yell</em> so that <code>root</code>'s equality check passes. (The number that appears after <code>humn:</code> in your input is now irrelevant.)</p>
<p>In the above example, the number you need to yell to pass <code>root</code>'s equality test is <code><em>301</em></code>. (This causes <code>root</code> to get the same number, <code>150</code>, from both of its monkeys.)</p>
<p><em>What number do you yell to pass <code>root</code>'s equality test?</em></p>
</article>


In [31]:
from more_itertools import one
import sympy as sym
from sympy.parsing.sympy_parser import parse_expr


tests = [
    {
        "name": "Example 1",
        "s": """
            root: pppw + sjmn
            dbpl: 5
            cczh: sllz + lgvd
            zczc: 2
            ptdq: humn - dvpt
            dvpt: 3
            lfqf: 4
            humn: 5
            ljgn: 2
            sjmn: drzm * dbpl
            sllz: 4
            pppw: cczh / lfqf
            lgvd: ljgn * ptdq
            drzm: hmdt - zczc
            hmdt: 32
        """,
        "expected": 301,
    },
]


class MonkeysII(Monkeys):
    def symbolic(self, node) -> int:
        def create_formula(node):
            if "value" in self.g[node]:
                return self.g[node]["value"] if node != "humn" else "x"
            left, right = self.g[node]["dependencies"]
            op = self.g[node]["operation"]

            self.g[node][
                "formula"
            ] = f"({create_formula(left)}{op}{create_formula(right)})"
            return self.g[node]["formula"]

        left, right = map(create_formula, self.g[node]["dependencies"])

        equals = eval(left if "x" not in left else right)
        equation = right if "x" not in left else left

        x = sym.Symbol("x")
        equation = sym.Eq(parse_expr(equation), equals)
        return one(int(i) for i in sym.solve(equation, x))


@test(tests=tests)
def test_part_II(s: str) -> int:
    return MonkeysII(s).symbolic("root")


Test Example 1 passed, for test_part_II.
Success


In [32]:
print(f"PartI II: {MonkeysII(puzzle).symbolic('root')}")

PartI II: 3769668716709


<link href="style.css" rel="stylesheet"></link>

<main>

<p>Your puzzle answer was <code>3769668716709</code>.</p><p class="day-success">Both parts of this puzzle are complete! They provide two gold stars: **</p>

</main>
